# [모듈 4.1] 모델 훈련 스텝 개발 (SageMaker Model Building Pipeline 훈련 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 0. 모델 훈련 개요 
- 1. 데이터 세트 로딩 및 기본 훈련 변수 설정
- 2. 모델 훈련 코드 확인
- 3. 모델 훈련 스텝 개발 및 실행
    - 아래의 3단계를 진행하여 SageMaker Model Building Pipeline 에서 훈련 스텝 개발 함. 아래의 (1), (2) 단계는 옵션이지만, 실제 현업 개발시에 필요한 단계이기에 실행을 권장 드립니다.
        - (1) [옵션] **[로컬 노트북 인스턴스]**에서 다커 컨테이너로 훈련 코드 실행 (로컬 모드로 불리움)
        - (2) [옵션] **[세이지 메이커 호스트 모드]**로 (로컬 노트북 인스턴스에서 실행이 되는 것이 아님) 다커 컨테이너를 통해서 훈련 코드 실행          
        - (3) [필수] SageMaker Model Building Pipeline 에서 모델 훈련 스텝 개발 및 실행
    
---
### 노트북 커널
- 이 워크샵은 노트북 커널이 `conda_python3` 를 사용합니다. 다른 커널일 경우 변경 해주세요.
---



# 0. 모델 훈련 개요

이 노트북은 세이지 메이커의 Training Job을 통해서 모델 훈련을 합니다. <br>
상세한 사항은 개발자 가이드를 참조 하세요. -->  [모델 훈련](https://sagemaker.readthedocs.io/en/stable/overview.html#local-mode)

- 일반적으로 크게 4가지의 스텝으로 진행이 됩니다.
    - (1) S3에 훈련 파일 준비
        - 이전 단계의 전처리시의 결과 파일을 사용 합니다.
    - (2) 훈련 알고리즘을 준비 (세이지 메이커 내장 알고리즘 혹은 사용자 정의 알고리즘 사용)
        - 이 노트북에서는 사용자 정의 알고리즘을 코드로 기술 했습니다. (훈련 스크립트 모드)
    - (3) Training Job을 생성시에 아래와 같은 항목을 제공합니다.
        - Training Job을 실행할 EC2(예: ml.m4.2xlarge) 기술
        - EC2에서 로딩할 다커 이미지의 이름 기술
            - 아래 그림의 ECR(Amazon Elastic Container Registry)에서 다운로드
        - S3 입력 파일 경로
        - 훈련 코드 경로
        - 훈련 결과로서 모델 아티펙트 S3 경로 (지정하지 않으면 디폴트를 사용 합니다.)
    - (4) EC2에서 훈련 실행 하여 S3에 모델 아티펙트 저장


![sagemaker-training.png](img/sagemaker-training.png)

# 1.데이터 세트 로딩 및 기본 훈련 변수 설정

In [1]:
import boto3
import sagemaker
import pandas as pd
import os

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()


%store -r 
# 노트북에 저장되어 있는 변수를 보기 위해서는 주석을 제거하고 실행하시면 됩니다.
# %store  

## 데이터 세트 로딩
- 이전 단계(전처리)에서 결과 파일을 로딩 합니다. 실제 훈련에 제공되는 데이터를 확인하기 위함 입니다.
- 로딩힐 데이터 파일이 S3에 있는지 변수의 경로를 확인 합니다. (train_preproc_dir_artifact)

In [2]:
! aws s3 ls {train_preproc_dir_artifact}

                           PRE train/


In [3]:
train_preproc_dir_artifact_file = os.path.join(train_preproc_dir_artifact,'train.csv')
print("train_preproc_dir_artifact_file: \n", train_preproc_dir_artifact_file)
train_prep_df = pd.read_csv(train_preproc_dir_artifact_file)
train_prep_df

train_preproc_dir_artifact_file: 
 s3://sagemaker-ap-northeast-2-057716757052/sklearn-fraud-process-2021-08-02-14-20-04-608/output/train/train.csv


,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
0,0,8913.668763,80513.668763,54,94,0,1,750,3000,99207,...,0,0,1,0,0,0,1,0,1,0
1,0,19746.724395,26146.724395,41,165,0,1,750,2950,95632,...,0,0,0,1,0,0,0,1,0,1
2,0,11652.969918,22052.969918,57,155,0,1,750,3000,93203,...,0,0,1,0,0,0,0,1,0,1
3,0,11260.930936,115960.930936,39,80,0,1,750,3000,85208,...,0,0,1,0,0,0,1,0,1,0
4,0,27987.704652,31387.704652,39,60,0,1,750,3000,91792,...,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,18052.611626,67152.611626,42,103,1,1,750,3000,93654,...,0,0,1,0,0,0,1,0,1,0
3996,0,34949.202468,51749.202468,23,6,0,3,750,3000,94305,...,0,0,0,1,1,0,0,0,1,0
3997,0,4063.701410,9963.701410,44,35,0,2,750,2550,95476,...,0,0,1,0,0,0,0,1,0,1
3998,0,17390.520451,20490.520451,22,38,0,1,750,3000,90680,...,0,1,0,0,0,0,0,1,0,1


### 기본 훈련 변수 및 하이퍼파라미터 설정
- XGBoost에 알고리즘에 입력될 하이퍼 파리미터의 값을 설정 합니다.
- scale_pos_weight 의 경우는 현재의 데이터가 레이블(fraud)간 불균형이 있기에, fraud: 1, non-fraud: 0 의 비율을 계산하여 제공합니다.
    - 하이퍼 파라미터 상세 사항은 여기를 보세요. -->  [XGBoost Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html)

In [4]:
def get_pos_scale_weight(df, label):
    '''
    1, 0 의 레이블 분포를 계산하여 클래스 가중치 리턴
    예: 1: 10, 0: 90 이면 90/10 = 9 를 제공함. 
    호출:
        class_weight = get_pos_scale_weight(train_prep_df, label='fraud')
    '''
    fraud_sum = df[df[label] == 1].shape[0]
    non_fraud_sum = df[df[label] == 0].shape[0]
    class_weight = int(non_fraud_sum / fraud_sum)
    print(f"fraud_sum: {fraud_sum} , non_fraud_sum: {non_fraud_sum}, class_weight: {class_weight}")
    return class_weight
    
class_weight = get_pos_scale_weight(train_prep_df, label='fraud')

fraud_sum: 131 , non_fraud_sum: 3869, class_weight: 29


In [5]:
from sagemaker.xgboost.estimator import XGBoost

bucket = sagemaker_session.default_bucket()
prefix = project_prefix

estimator_output_path = f's3://{bucket}/{prefix}/training_jobs'
train_instance_count = 1

hyperparameters = {
       "scale_pos_weight" : class_weight,    
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}
%store hyperparameters

Stored 'hyperparameters' (dict)


# 2. 훈련 스크립트 확인

전처리 코드는 크게 아래와 같이 구성 되어 있습니다.
- 커맨드 인자로 전달된 변수 내용 확인
- 훈련 데이터를 로딩 및 하이퍼파라미터 설정
- xgboost의 cross-validation(cv) 로 훈련 합니다.
- Cross-Validation으로 훈련하여, 훈련 및 검증 메트릭 추출
- 훈련 및 검증 데이터 세트의 roc-auc 값을 metrics_data 에 저장
- 오직 훈련 데이터 만으로 훈련하여 모델 생성
- 모델 아티펙트 및 훈련/검증 지표를 저장
    - [알림] 일반적으로 xgboost의 알고리즘의 큰 변경이 없으면, 세이지 메이커 내장 xgboost 알고리즘을 사용합니다. 여기서는 훈련 코드를 사용자가 정의해서 사용할 수 있는 예시를 위하여 따로 훈련 코드를 만들었습니다.
    - 참고:  [세이지메이커 XGBoost 알고리즘](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/xgboost.html)
---

In [6]:
!pygmentize src/xgboost_starter_script.py

import os
import sys
import pickle
import xgboost as xgb
import argparse
import pandas as pd
import json

import pandas as pd
pd.options.display.max_rows=20
pd.options.display.max_columns=10

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    ###################################
    ## 커맨드 인자 처리
    ###################################    
    
    # Hyperparameters are described here
    parser.add_argument('--scale_pos_weight', type=int, default=50)    
    parser.add_argument('--num_round', type=int, default=999)
    parser.add_argument('--max_depth', type=int, default=3)
    parser.add_argument('--eta', type=float, default=0.2)
    parser.add_argument('--objective', type=str, default='binary:logistic')
    parser.add_argument('--nfold', type=int, default=5)
    parser.add_argument('--early_stopping_rounds', type=int, default=10)
    parser.add_argument('--train_data_path', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))

    # SageMaker specific arguments. D

# 3. 모델 훈련 스텝 개발 및 실행

## 3.1 개요

### (1) **[로컬 노트북 인스턴스]**에서 다커 컨테이너로 훈련 코드 실행 (로컬 모드로 불리움)
- 이 단계의 목적은 현업 프로젝트의 개발시에 로컬 노트북 인스턴스의 다커 컨테이터 안에서 로직 확인, 디버깅이 수월하기 때문에 단계를 진행 합니다. 
    - 처음 실행시에 다커 이미지를 로컬 노트북 인스턴스에 다운로드 받는데에 1-2분 소요 됩니다. 
    - 로컬 노트북 인스턴스의 다커 컨테이너로 실행하기 이전에, 쥬피터 노트북에서 훈련 코드를 실행할 수 있습니다. 이 노트북에서는 이 과정을 생략 했습니다.
- [알림] 로컬 모드 참고 자료
    - 로컬모드 설명하는 블로그 자료 --> [Use the Amazon SageMaker local mode to train on your notebook instance](https://aws.amazon.com/blogs/machine-learning/use-the-amazon-sagemaker-local-mode-to-train-on-your-notebook-instance/)
    - TF, Pytorch, SKLean, SKLearn Processing JOb에 대한 로컬 모드 샘플 --> [Amazon SageMaker Local Mode Examples](https://github.com/aws-samples/amazon-sagemaker-local-mode)
    - Python SDK -->  [로컬모드 Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html#local-mode)
    - [SageMaker 에서 도커 컨테이너 사용하기](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/docker-containers.html)
    - [Hello 다커](https://github.com/mullue/hello-docker/blob/master/hello_docker.ipynb)

    
### (2) **[세이지 메이커 호스트 모드]** 로 (**로컬 노트북 인스턴스에서 실행이 되는 것이 아님**) 다커 컨테이너를 통해서 훈련 코드 실행      
- 위의 (1) 단계에서 훈련 코드의 로직 확인이 되었기에, 실제 세이지 메이커의 호스트 모드로 다커 컨테이너를 통해 훈련 코드를 수행 합니다.
- 이 단계의 목적은 SageMaker Model Building Pipeline이 다커 컨테이너 형태로 실행이 되기에, 이 단계에서 다커 컨테이너에서 정상적으로 동작하는지를 확인하기 위해서 이 단계를 수행 합니다.

    
### (3) SageMaker Model Building Pipeline 에서 훈련을 수행합니다.
- 상세 사항은 여기에서 확인 하세요. --> [Amazon SageMaker 모델 구축 파이프라인](img/https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)
    
---    
    
## 3.2 실행
### (1) **[로컬 노트북 인스턴스]**에서 다커 컨테이너로 훈련 코드 실행 (로컬 모드로 불리움)
- 최초 실행시에 약 1-3분 소요 됩니다.
    - 다운로드가 끝나고 아래와 같이 터미널에서 `docker image ls` 명령어 실행하시면 `366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost` 가 다운로드 되신 것을 확인할 수 있습니다.

```
sh-4.2$ docker image ls
REPOSITORY                                                                 TAG                 IMAGE ID            CREATED             SIZE
366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn   0.23-1-cpu-py3      17d4d9baed63        4 months ago        3.36GB
366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost        1.0-1-cpu-py3       581593966bd9        4 months ago        3.35GB
```

- `instance_type=local` 를 제공해서 로컬 모드로 동작합니다. 

#### [참조] 로컬모드로 실행시에 403 Forbidden Error 발생시
- [로컬 모드 실행시 403 에러 해결 방법](error/Trouble-Shooting.ipynb)

In [7]:

xgb_estimator = XGBoost(
    entry_point = "xgboost_starter_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    sagemaker_session = sagemaker.LocalSession(),
    instance_count = train_instance_count,
    instance_type = 'local',
    framework_version = "1.0-1")

In [8]:
xgb_estimator.fit(inputs = {'train': train_preproc_dir_artifact})


Creating uxp2gwe8w0-algo-1-1q58y ... 
Creating uxp2gwe8w0-algo-1-1q58y ... done
Attaching to uxp2gwe8w0-algo-1-1q58y
uxp2gwe8w0-algo-1-1q58y | INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
uxp2gwe8w0-algo-1-1q58y | INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
uxp2gwe8w0-algo-1-1q58y | INFO:sagemaker_xgboost_container.training:Invoking user training script.
uxp2gwe8w0-algo-1-1q58y | INFO:sagemaker-containers:Module xgboost_starter_script does not provide a setup.py. 
uxp2gwe8w0-algo-1-1q58y | Generating setup.py
uxp2gwe8w0-algo-1-1q58y | INFO:sagemaker-containers:Generating setup.cfg
uxp2gwe8w0-algo-1-1q58y | INFO:sagemaker-containers:Generating MANIFEST.in
uxp2gwe8w0-algo-1-1q58y | INFO:sagemaker-containers:Installing module with the following command:
uxp2gwe8w0-algo-1-1q58y | /miniconda3/bin/python3 -m pip install . 
uxp2gwe8w0-algo-1-1q58y | Processing /opt/ml/code
uxp2gwe8w0-algo-1-1q58y | Building wheels for collected

Failed to delete: /tmp/tmpbd0fqi7q/algo-1-1q58y Please remove it manually.


uxp2gwe8w0-algo-1-1q58y exited with code 0
Aborting on container exit...
===== Job Complete =====


### (2) 세이지메이커 호스트 모드(로컬 다커 컨테이너 사용)로 훈련 코드 실행
- `instance_type = 'ml.m5.xlarge'` 를 제공해서 ml.m5.xlarge EC2 인스턴스를 생성하고, 아 안에서 모델 훈련이 진행 됩니다.
- `xgb_estimator.fit(inputs = {'train': train_preproc_dir_artifact}, wait=False)` 에서 wait=False 를 제공했기 때문에 노트북에서는 비동기로 진행합니다. 즉 셀이 완료까지 기다리지 않고 다음 셀의 실행이 가능합니다.

In [9]:
instance_type = 'ml.m5.xlarge'


xgb_estimator = XGBoost(
    entry_point = "xgboost_starter_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = train_instance_count,
    instance_type = instance_type,
    framework_version = "1.0-1")

xgb_estimator.fit(inputs = {'train': train_preproc_dir_artifact}, wait=False)


#### 훈련 잡을 SageMaker Console에서 확인
- 실제 SageMaker Console에 이동하셔서 훈련 잡이 실행 중인지 확인 해보세요.
![train_job_console.png](img/train_job_console.png)

estimator의 logs() 함수를 통해서 훈련잡의 로그를 확인할 수 있습니다. 

In [10]:
xgb_estimator.logs()

2021-08-03 05:00:05 Starting - Starting the training job...
2021-08-03 05:00:28 Starting - Launching requested ML instancesProfilerReport-1627966804: InProgress
...
2021-08-03 05:00:59 Starting - Preparing the instances for training.........
2021-08-03 05:02:34 Downloading - Downloading input data
2021-08-03 05:02:34 Training - Downloading the training image...
2021-08-03 05:02:58 Uploading - Uploading generated training modelINFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module xgboost_starter_script does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO:sagemaker-containers:Generating MANIFEST.in
INFO:sagemaker-containers:Installing module with the following command:
/miniconda3/bin/python3 -m pip install . 
Processing /opt/ml/code
  

### (3) SageMaker Pipeline에서  실행 
---



### 모델 빌딩 파이프라인 변수 생성



In [11]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_count = ParameterInteger(
    name="TrainingInstanceCount",
    default_value= 1
)

input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)


### 모델 학습을 위한 학습단계 정의 

본 단계에서는 SageMaker의 [XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) 알고리즘을 이용하여 학습을 진행할 것입니다. XGBoost 알고리즘을 이용하도록 Estimator를 구성합니다. 보편적인 학습스크립트를 이용하여 입력 채널에서 정의한 학습데이터를 로드하고, 하이퍼파라미터 설정을 통해 학습을 설정하고, 모델을 학습한 후 `model_dir`경로에 학습된 모델을 저장합니다. 저장된 모델은 이후 호스팅을 위해 사용됩니다. 

학습된 모델이 추출되어 저장될 경로 또한 명시되었습니다. 

`training_instance_type`파라미터가 사용된 것을 확인합니다. 이 값은 본 예제의 파이프라인에서 여러번 사용됩니다. 본 단계에서는 estimator를 선언할 때 전달되었습니다. 


In [12]:
xgb_train = XGBoost(
    entry_point = "xgboost_starter_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = training_instance_count,
    instance_type = training_instance_type,
    framework_version = "1.0-1")

이전 단계에서 (프로세싱) 전처리 훈련, 검증 데이터 세트를 입력으로 제공 합니다.
- [알림] `8.5.All-Pipeline.ipynb` 노트북에서는 입력을 전처리 스텝의 결과를 지정합니다. 여기서는 전처리 스텝과 독립적으로 실행하기 위해서 S3의 입력 파일 경로를 직접 기술 하였습니다.
-  `8.5.All-Pipeline.ipynb` 에서의 step_train 코드

```python
step_train = TrainingStep(
    name="FraudScratchTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
    },
)    
 ```


In [13]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="FraudScratchTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data= train_preproc_dir_artifact,
            content_type="text/csv"
        ),
    },
)

### 모델 빌딩 파이프라인 정의
- 위에서 정의한 파라미터와, 단계에는 step_train 를 기술합니다.

In [14]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        training_instance_type,        
        training_instance_count,         
        input_data,
    ],
    steps=[step_train],
)

In [15]:
import json

definition = json.loads(pipeline.definition())
# definition

#### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 SageMaker Pipelines 서비스에 제출하여 파이프라인을 생성하거나 파이프라인이 이미 존재하면 파이프라인 정의를 업데이트합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [16]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step',
 'ResponseMetadata': {'RequestId': '9a2d3e03-73be-4b4c-8414-dc019e720dbd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9a2d3e03-73be-4b4c-8414-dc019e720dbd',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '104',
   'date': 'Tue, 03 Aug 2021 05:03:47 GMT'},
  'RetryAttempts': 0}}

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [17]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [18]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step/execution/deob50sfby30',
 'PipelineExecutionDisplayName': 'execution-1627967028205',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 8, 3, 5, 3, 48, 138000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 8, 3, 5, 3, 48, 138000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '85609b9f-78a2-4602-9001-8051a1579336',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '85609b9f-78a2-4602-9001-8051a1579336',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '437',
   'date': 'Tue, 03 Aug 2021 05:03:47 GMT'},
  'RetryAttempts': 0}}

In [19]:

execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [20]:
execution.list_steps()

[{'StepName': 'FraudScratchTrain',
  'StartTime': datetime.datetime(2021, 8, 3, 5, 3, 48, 771000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 3, 5, 6, 51, 490000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:training-job/pipelines-deob50sfby30-fraudscratchtrain-gyxi8ygrwl'}}}]

### [옵션] SageMaker Studio에서 확인하기
- 이전의 3.1.Preprocesing-Pipleline 노트북에서 언급 되었듯이, SageMaker Studio 에서도 확인이 가능합니다. 이전 노트북을 참조 해주세요.


### 아티펙트 경로 추출
위의 훈련 스텝이 완료되면 실행해주세요

In [21]:
def get_train_artifact(execution, client, job_type,  kind=0):
    '''
    kind: 0 --> train
    kind: 2 --> test
    '''
    response = execution.list_steps()
    # print("response: ", response)
    proc_arn = response[0]['Metadata'][job_type]['Arn']
    train_job_name = proc_arn.split('/')[-1]
    # print("train_job_name: ", train_job_name)
    response = client.describe_training_job(TrainingJobName = train_job_name)
    # print("\nresponse: ", response)    
    train_model_artifact = response['ModelArtifacts']['S3ModelArtifacts']    
    
    return train_model_artifact

import boto3
client = boto3.client("sagemaker")
    
train_model_artifact = get_train_artifact(execution, client,job_type='TrainingJob', kind=0)
print(" train_model_artifact: ", train_model_artifact)


 train_model_artifact:  s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step/training_jobs/pipelines-deob50sfby30-FraudScratchTrain-GyXI8YgRWL/output/model.tar.gz


In [22]:
image_uri = xgb_train.image_uri
print("image_uri: \n", image_uri)

image_uri: 
 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3


훈련 모델 아티펙트와, 훈련시 사용한 다커 이미지의 경로를 저장 합니다.

In [23]:
%store train_model_artifact
%store image_uri

Stored 'train_model_artifact' (str)
Stored 'image_uri' (str)
